<a href="https://www.kaggle.com/code/elcientfico/driving-a-taxi-using-tree-methods-and-convolutions?scriptVersionId=149217955" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
NOTE: This experiment failed, you can continue to try, I know I will, just now now.

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import (Input,Conv2D,GlobalMaxPooling2D,Conv3D,
                                    GlobalMaxPooling3D)# Input data files are available in the readGlobalMaxPooling3D-only "../input/" directory
from tensorflow.keras.models import Model# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from PIL import Image
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [17]:
im_size = Image.open("/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots/1000_1010.png").size
im_size = (im_size[1],im_size[0],3)

In [113]:
class conv_model:
    def __init__(self):
        pass
def conv_model(ishape = (3,512,512,3), filters = 256, kernel_s = 3, strides = 3):
    x1  = Input(shape = ishape)
    x2 = Conv3D(filters,kernel_s,strides = (1,strides,strides), padding  = "same")(x1)
    x2 = Conv3D(filters*2,kernel_s,strides = (1,strides,strides), padding  = "same")(x2)
    x2 = Conv3D(filters*4,kernel_s,strides = (1,strides,strides), padding  = "same")(x2)
    x2 = Conv3D(filters*8,kernel_s,strides = (1,strides,strides), padding  = "same")(x2)
    x2 = Conv3D(filters*16,kernel_s,strides = (1,strides,strides), padding  = "same")(x2)
    x4 = GlobalMaxPooling3D()(x2)
    ppv = tf.expand_dims(tf.reduce_mean(tf.cast(tf.math.greater(x2,0),dtype = tf.float32),axis = [1,2,3,4]),-1)
    x4 = tf.concat((x4,ppv),-1)
    model = Model(x1,x4)
    return model


In [118]:
n_steps = 6
input_shape = (n_steps,*im_size)
model = conv_model(input_shape,64,3,4)
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_33 (InputLayer)          [(None, 6, 600, 795  0           []                               
                                , 3)]                                                             
                                                                                                  
 conv3d_363 (Conv3D)            (None, 6, 150, 199,  5248        ['input_33[0][0]']               
                                 64)                                                              
                                                                                                  
 conv3d_364 (Conv3D)            (None, 6, 38, 50, 1  221312      ['conv3d_363[0][0]']             
                                28)                                                        

In [119]:
ss_path = "/kaggle/input/gta-san-andreas-driving-dataset-with-label/GTA_SanAndreasDriving/screenshots"
file_list = os.listdir(ss_path)
keys = ["W","S","A","D"]
keys_list = np.array([list(file_name.split("_")[1].split(".")[0]) for file_name in file_list]).astype(int)

LAod images

In [120]:
def get_batch(sub_file_list,ss_path,steps_n):
    steps_n = steps_n-1
    image_batch = []
    for idx,file_name in enumerate(sub_file_list):
        img = np.array(Image.open(os.path.join(ss_path,file_name)))
        image_batch.append(img)
    image_batch = np.stack(image_batch)
    image_batch = np.expand_dims(image_batch,1)
    aditional_arrays = None
    for step in range(steps_n):
        shifted_array = image_batch[:-step]
        zeros_array = np.zeros_like(image_batch[-step:])
        shifted_array = np.concatenate((zeros_array,shifted_array),0)
        assert(shifted_array.shape == image_batch.shape)
        if step == 0:
            aditional_arrays = shifted_array
        else:
            aditional_arrays = np.concatenate((aditional_arrays,shifted_array),1)
    image_batch = np.concatenate((image_batch,aditional_arrays),1)
    return image_batch

In [121]:
from tqdm import tqdm
enc_img_list = []
batch_size = 4
for batch_n in tqdm(range(np.ceil(len(file_list)/batch_size).astype(int))):
    curr_range = slice((batch_n*batch_size),((batch_n+1)*batch_size))
    sub_file_list = file_list[curr_range]
    curr_batch = get_batch(sub_file_list,ss_path,n_steps)
    curr_pred = model.predict_on_batch(curr_batch)
    enc_img_list.append(curr_pred)
enc_img_list = np.concatenate(enc_img_list,0)
# for image_path in tqdm(file_list):
#     im = np.array(Image.open(os.path.join(ss_path,image_path)))
#     im = np.expand_dims(im/255,0)
#     conv_im = model.predict(im,verbose = False)
#     enc_img_list.append(conv_im)


100%|██████████| 1428/1428 [04:08<00:00,  5.74it/s]


In [122]:
enc_img_list = np.array(enc_img_list).squeeze()
keys_list = np.array(keys_list)

In [123]:
import xgboost as xgb

In [124]:
#["W","S","A","D"]

def mullab_to_mulclass(y):
    if all(y == [1,0,0,0]):
        return [1,0,0,0,0,0,0,0,0]
    elif all(y == [0,1,0,0]):
        return [0,1,0,0,0,0,0,0,0]
    elif all(y == [0,0,1,0]):
        return [0,0,1,0,0,0,0,0,0]
    elif all(y == [0,0,0,1]):
        return [0,0,0,1,0,0,0,0,0]
    elif all(y == [1,1,0,0]):
        return [0,0,0,0,1,0,0,0,0]
    elif all(y == [1,0,1,0]):
        return [0,0,0,0,0,1,0,0,0]
    elif all(y == [0,1,1,0]):
        return [0,0,0,0,0,0,1,0,0]
    elif all(y == [0,0,1,1]):
        return [0,0,0,0,0,0,0,1,0]
    else:
        return [0,0,0,0,0,0,0,0,1]
ylabels = []
for label in keys_list.astype(int):
    ylabels.append(mullab_to_mulclass(label))

In [125]:
prev_steps = 20
enc_img_list_plus_prev = np.array(enc_img_list)
for step in range(1,prev_steps+1):
    enc_img_list_0 = np.concatenate(
        (enc_img_list[:-step],np.zeros(shape = enc_img_list[:step,:].shape))
        ,0
    )
    enc_img_list_plus_prev = np.concatenate((enc_img_list_plus_prev,enc_img_list_0),axis = -1)

In [126]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(enc_img_list,keys_list,test_size=.1,stratify = keys_list) 

In [128]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
# rf = MultiOutputClassifier(HistGradientBoostingClassifier(early_stopping = True, verbose  = True,max_iter = 10000, learning_rate = 0.01,
#                                                          n_iter_no_change = 100, scoring = "f1",min_samples_leaf = 1, 
#                                                           max_leaf_nodes = 14))

    
early_stop = xgb.callback.EarlyStopping(
rounds=5, metric_name='logloss', save_best=True,
)

print("traing...")

rf = xgb.XGBClassifier(n_estimators = 10000,callbacks=[early_stop], learning_rate = .1, eval_metric = "logloss")

rf.fit(x_train,y_train[:,1:],eval_set=[(x_test, y_test[:,1:])], verbose = True)

print("Test Acc",rf.score(x_test,y_test[:,1:]),
"Train Acc",rf.score(x_train,y_train[:,1:]))

print("Test F1",f1_score(y_test[:,1:],rf.predict(x_test),average = "micro"),
"Train F1",f1_score(y_train[:,1:],rf.predict(x_train),average = "micro"))


traing...
[0]	validation_0-logloss:0.64166
[1]	validation_0-logloss:0.60002
[2]	validation_0-logloss:0.56559
[3]	validation_0-logloss:0.53646
[4]	validation_0-logloss:0.51296
[5]	validation_0-logloss:0.49276
[6]	validation_0-logloss:0.47533
[7]	validation_0-logloss:0.46120
[8]	validation_0-logloss:0.44844
[9]	validation_0-logloss:0.43800
[10]	validation_0-logloss:0.42905
[11]	validation_0-logloss:0.42175
[12]	validation_0-logloss:0.41519
[13]	validation_0-logloss:0.41018
[14]	validation_0-logloss:0.40544
[15]	validation_0-logloss:0.40123
[16]	validation_0-logloss:0.39751
[17]	validation_0-logloss:0.39479
[18]	validation_0-logloss:0.39170
[19]	validation_0-logloss:0.38977
[20]	validation_0-logloss:0.38797
[21]	validation_0-logloss:0.38641
[22]	validation_0-logloss:0.38482
[23]	validation_0-logloss:0.38357
[24]	validation_0-logloss:0.38307
[25]	validation_0-logloss:0.38240
[26]	validation_0-logloss:0.38095
[27]	validation_0-logloss:0.37973
[28]	validation_0-logloss:0.37976
[29]	validatio

In [ ]:
df_pred_x_train = pd.DataFrame(rf.predict(x_train))

In [ ]:
key_strokes = [np.array(keys)[pred.astype(bool)] for pred in rf.predict(x_train)]